In [1]:
(require booleans redex/reduction-semantics)

In [2]:
(define/ppl start #:no-check
  (;; nothing
   (r0 = GO)
   (r1 = false)
   (rsel = false)
   ;; pause
   (l1 = GO)
   (l0 = (and RES reg-out))
   (lsel = reg-out)
   (reg-in = (and (not KILL) reg-set))
   (reg-set = (or is-susp GO))
   (is-susp = (and SUSP reg-out))
   (SEL = (or rsel lsel))
    ;; synchronizer
    (K1 = (and left1 (and both1 right1)))
    (K0 = (and left0 (and both0 right0)))
    (left1 = (or left0 l1))
    (right1 = (or right0 r1))
    (both1 = (or l1 r1))
    (left0 = (or l0 lem))
    (right0 = (or r0 rem))
    (both0 = (or l0 r0))
    (rem = (and SEL (and RES (not rsel))))
    (lem = (and SEL (and RES (not lsel))))))
    

(r0 = GO)
(r1 = false)
(rsel = false)
(l1 = GO)
(l0 = (and RES reg-out))
(lsel = reg-out)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp GO))
(is-susp = (and SUSP reg-out))
(SEL = (or rsel lsel))
(K1 = (and left1 (and both1 right1)))
(K0 = (and left0 (and both0 right0)))
(left1 = (or left0 l1))
(right1 = (or right0 r1))
(both1 = (or l1 r1))
(left0 = (or l0 lem))
(right0 = (or r0 rem))
(both0 = (or l0 r0))
(rem = (and SEL (and RES (not rsel))))
(lem = (and SEL (and RES (not lsel))))


In [3]:
(define/ppl end #:no-check
   ((K1 = GO)
   (K0 = (and RES reg-out))
   (SEL = reg-out)
   (reg-in = (and (not KILL) reg-set))
   (reg-set = (or is-susp GO))
   (is-susp = (and SUSP reg-out))))

(K1 = GO)
(K0 = (and RES reg-out))
(SEL = reg-out)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp GO))
(is-susp = (and SUSP reg-out))


In [4]:
(assert-same (term start) (term end))

In [5]:
(assert-same (term (replace* start [GO ⊥] [reg-out false]))
             (term (replace* end [GO ⊥] [reg-out false])))

In [6]:
(define/ppl step1 #:no-check
  (convert-P (replace* start [GO ⊥] [reg-out false])))

((+ r0) = false)
((- r0) = false)
((+ r1) = false)
((- r1) = true)
((+ rsel) = false)
((- rsel) = true)
((+ l1) = false)
((- l1) = false)
((+ l0) = (and (+ RES) false))
((- l0) = (or (- RES) true))
((+ lsel) = false)
((- lsel) = true)
((+ reg-in) = (and (- KILL) (+ reg-set)))
((- reg-in) = (or (+ KILL) (- reg-set)))
((+ reg-set) = (or (+ is-susp) false))
((- reg-set) = (and (- is-susp) false))
((+ is-susp) = (and (+ SUSP) false))
((- is-susp) = (or (- SUSP) true))
((+ SEL) = (or (+ rsel) (+ lsel)))
((- SEL) = (and (- rsel) (- lsel)))
((+ K1) = (and (+ left1) (and (+ both1) (+ right1))))
((- K1) = (or (- left1) (or (- both1) (- right1))))
((+ K0) = (and (+ left0) (and (+ both0) (+ right0))))
((- K0) = (or (- left0) (or (- both0) (- right0))))
((+ left1) = (or (+ left0) (+ l1)))
((- left1) = (and (- left0) (- l1)))
((+ right1) = (or (+ right0) (+ r1)))
((- right1) = (and (- right0) (- r1)))
((+ both1) = (or (+ l1) (+ r1)))
((- both1) = (and (- l1) (- r1)))
((+ left0) = (or (+ l0) (+ lem)

In [7]:
(define/ppl step2 
  propigate/remove* step1 (+ r0) (- r0) (- l1) (+ l1) (+ SEL) (- SEL) (+ l0) (- l0))

((+ r1) = false)
((- r1) = true)
((+ rsel) = false)
((- rsel) = true)
((+ lsel) = false)
((- lsel) = true)
((+ reg-in) = (and (- KILL) (+ reg-set)))
((- reg-in) = (or (+ KILL) (- reg-set)))
((+ reg-set) = (or (+ is-susp) false))
((- reg-set) = (and (- is-susp) false))
((+ is-susp) = (and (+ SUSP) false))
((- is-susp) = (or (- SUSP) true))
((+ K1) = (and (+ left1) (and (+ both1) (+ right1))))
((- K1) = (or (- left1) (or (- both1) (- right1))))
((+ K0) = (and (+ left0) (and (+ both0) (+ right0))))
((- K0) = (or (- left0) (or (- both0) (- right0))))
((+ left1) = (or (+ left0) false))
((- left1) = (and (- left0) false))
((+ right1) = (or (+ right0) (+ r1)))
((- right1) = (and (- right0) (- r1)))
((+ both1) = (or false (+ r1)))
((- both1) = (and false (- r1)))
((+ left0) = (or (and (+ RES) false) (+ lem)))
((- left0) = (and (or (- RES) true) (- lem)))
((+ right0) = (or false (+ rem)))
((- right0) = (and false (- rem)))
((+ both0) = (or (and (+ RES) false) false))
((- both0) = (and (or (- RE

In [8]:
(define/ppl step3 
  replace* step2
  [(and (+ RES) false) false]
  [(or (- RES) true) true]
  [(or (+ is-susp) false) (+ is-susp)]
  [(and (- is-susp) false) false]
  [(and (+ SUSP) false) false]
  [(or (- SUSP) true) true]
  [(or (+ left0) false) (+ left0)]
  [(and (- left0) false) false]
  [(or (+ right0) false) (+ right0)]
  [(and (- right0) true) (- right0)]
  [(or false false) false]
  [(and false true) false]
  [(and false false) false]
  [(or (+ l0) false) (+ l0)]
  [(and (- l0) false) false]
  [(and true false) false])

((+ r1) = false)
((- r1) = true)
((+ rsel) = false)
((- rsel) = true)
((+ lsel) = false)
((- lsel) = true)
((+ reg-in) = (and (- KILL) (+ reg-set)))
((- reg-in) = (or (+ KILL) (- reg-set)))
((+ reg-set) = (+ is-susp))
((- reg-set) = false)
((+ is-susp) = false)
((- is-susp) = true)
((+ K1) = (and (+ left1) (and (+ both1) (+ right1))))
((- K1) = (or (- left1) (or (- both1) (- right1))))
((+ K0) = (and (+ left0) (and (+ both0) (+ right0))))
((- K0) = (or (- left0) (or (- both0) (- right0))))
((+ left1) = (+ left0))
((- left1) = false)
((+ right1) = (or (+ right0) (+ r1)))
((- right1) = (and (- right0) (- r1)))
((+ both1) = (or false (+ r1)))
((- both1) = (and false (- r1)))
((+ left0) = (or false (+ lem)))
((- left0) = (and true (- lem)))
((+ right0) = (or false (+ rem)))
((- right0) = (and false (- rem)))
((+ both0) = false)
((- both0) = false)
((+ rem) = (and (or (+ rsel) (+ lsel)) (and (+ RES) (- rsel))))
((- rem) = (or (and (- rsel) (- lsel)) (or (- RES) (+ rsel))))
((+ lem) = (and (

In [9]:
(define/ppl step4
  propigate/remove* step3
  (+ reg-set) (- reg-set)
  (+ is-susp) (- is-susp) (+ right1) (- right1) (+ both1) (- both1) (+ left0) (- left0)
  (+ right0) (- right0) (+ both0) (- both0) (+ lem) (- lem))

((+ r1) = false)
((- r1) = true)
((+ rsel) = false)
((- rsel) = true)
((+ lsel) = false)
((- lsel) = true)
((+ reg-in) = (and (- KILL) false))
((- reg-in) = (or (+ KILL) false))
((+ K1)
 =
 (and (+ left1) (and (or false (+ r1)) (or (or false (+ rem)) (+ r1)))))
((- K1)
 =
 (or (- left1) (or (and false (- r1)) (and (and false (- rem)) (- r1)))))
((+ K0)
 =
 (and (or false (and (or (+ rsel) (+ lsel)) (and (+ RES) (- lsel))))
      (and false (or false (+ rem)))))
((- K0)
 =
 (or (and true (or (and (- rsel) (- lsel)) (or (- RES) (+ lsel))))
     (or false (and false (- rem)))))
((+ left1) = (or false (and (or (+ rsel) (+ lsel)) (and (+ RES) (- lsel)))))
((- left1) = false)
((+ rem) = (and (or (+ rsel) (+ lsel)) (and (+ RES) (- rsel))))
((- rem) = (or (and (- rsel) (- lsel)) (or (- RES) (+ rsel))))


In [10]:
(define/ppl step5
  replace* step4
  [(and (- KILL) false) false]
  [(or (+ KILL) false) (+ KILL)]
  [(and (+ left1) (and false false)) false]
  [(or (- left1) (or false false)) (- left1)]
  [(and (or false false) (and false false)) false]
  [(or (and true false) (or false false)) false]
  [(or false false) false])

((+ r1) = false)
((- r1) = true)
((+ rsel) = false)
((- rsel) = true)
((+ lsel) = false)
((- lsel) = true)
((+ reg-in) = false)
((- reg-in) = (+ KILL))
((+ K1)
 =
 (and (+ left1) (and (or false (+ r1)) (or (or false (+ rem)) (+ r1)))))
((- K1)
 =
 (or (- left1) (or (and false (- r1)) (and (and false (- rem)) (- r1)))))
((+ K0)
 =
 (and (or false (and (or (+ rsel) (+ lsel)) (and (+ RES) (- lsel))))
      (and false (or false (+ rem)))))
((- K0)
 =
 (or (and true (or (and (- rsel) (- lsel)) (or (- RES) (+ lsel))))
     (or false (and false (- rem)))))
((+ left1) = (or false (and (or (+ rsel) (+ lsel)) (and (+ RES) (- lsel)))))
((- left1) = false)
((+ rem) = (and (or (+ rsel) (+ lsel)) (and (+ RES) (- rsel))))
((- rem) = (or (and (- rsel) (- lsel)) (or (- RES) (+ rsel))))


In [11]:
(define/ppl step6
  propigate/remove* step5
  (+ reg-in) (- reg-in) (+ left1) (- left1))

((+ r1) = false)
((- r1) = true)
((+ rsel) = false)
((- rsel) = true)
((+ lsel) = false)
((- lsel) = true)
((+ K1)
 =
 (and (or false (and (or (+ rsel) (+ lsel)) (and (+ RES) (- lsel))))
      (and (or false (+ r1)) (or (or false (+ rem)) (+ r1)))))
((- K1) = (or false (or (and false (- r1)) (and (and false (- rem)) (- r1)))))
((+ K0)
 =
 (and (or false (and (or (+ rsel) (+ lsel)) (and (+ RES) (- lsel))))
      (and false (or false (+ rem)))))
((- K0)
 =
 (or (and true (or (and (- rsel) (- lsel)) (or (- RES) (+ lsel))))
     (or false (and false (- rem)))))
((+ rem) = (and (or (+ rsel) (+ lsel)) (and (+ RES) (- rsel))))
((- rem) = (or (and (- rsel) (- lsel)) (or (- RES) (+ rsel))))


In [12]:
(define/ppl step-a #:no-check
  (replace* end [GO ⊥] [reg-out false]))

(K1 = ⊥)
(K0 = (and RES false))
(SEL = false)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp ⊥))
(is-susp = (and SUSP false))


In [13]:
(define/ppl step-b
  replace* step-a
  [(and RES false) false]
  [(and SUSP false) false])

(K1 = ⊥)
(K0 = false)
(SEL = false)
(reg-in = (and (not KILL) reg-set))
(reg-set = (or is-susp ⊥))
(is-susp = false)
